In [45]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "OpenLLM-Ro/RoLlama3-8b-Instruct-4bit"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now default to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Initialize IR endpoints

In [46]:
from py4j.java_gateway import JavaGateway

# Connect to the Java Gateway
gateway = JavaGateway()
ir_system = gateway.entry_point

#### CREATE INDEX #####
# folderPath = "src/main/resources"
# ir_system.index(folderPath)
#### CREATE INDEX #####

# Tested queries and expected docs

In [47]:
# Test 1
query1 = "Ce pot vizita în România?"

# Expected documents:
docs1 = ["doc11.txt", "doc13.txt", "doc19.txt"]
# Test 2
query2 = "Care sunt mâncărurile tradiționale românești?"

# Expected documents:
docs2 = ["doc17.txt", "doc27.txt", "doc11.txt"]
# Test 3
query3 = "Cine sunt artiștii români celebri?"

# Expected documents:
docs3 = ["doc22.txt", "doc25.txt", "doc27.txt"]
# Test 4
query4 = "Ce știi despre Castelul Bran și Dracula?"

# Expected documents:
docs4 = ["doc23.txt", "doc21.txt", "doc19.txt"]
# Test 5
query5 = "Care este limbajul de programare utilizat în învățarea automată?"

# Expected documents:
docs5 = ["doc12.txt", "doc18.txt", "doc20.txt"]
# Test 6
query6 = "Care sunt râurile mari din România?"

# Expected documents:
docs6 = ["doc30.txt", "doc19.txt", "doc28.txt"]
# Test 7
query7 = "Care sunt limbajele de programare populare?"
# Expected documents:
docs7 = ["doc12.txt", "doc20.txt", "doc14.txt"]

# Test 8
query8 = "Ce poți vizita în Transilvania?"
# Expected documents:
docs8 = ["doc21.txt", "doc23.txt", "doc13.txt"]

# Test 9
query9 = "Cine sunt compozitorii celebri din România?"
# Expected documents:
docs9 = ["doc22.txt", "doc25.txt", "doc27.txt"]

# Test 10
query10 = "Ce înseamnă învățarea automată?"
# Expected documents:
docs10 = ["doc18.txt", "doc12.txt", "doc20.txt"]

# Test 11
query11 = "Care sunt tradițiile românești importante?"
# Expected documents:
docs11 = ["doc27.txt", "doc17.txt", "doc11.txt"]

# Test 12
query12 = "Ce este Bucureștiul cunoscut ca?"
# Expected documents:
docs12 = ["doc16.txt", "doc26.txt", "doc17.txt"]

# Test 13
query13 = "Ce castele sunt faimoase în România?"
# Expected documents:
docs13 = ["doc23.txt", "doc21.txt", "doc19.txt"]

# Test 14
query14 = "Ce lacuri faimoase există în România?"
# Expected documents:
docs14 = ["doc28.txt", "doc29.txt", "doc30.txt"]

# Test 15
query15 = "Cine a scris Rapsodia Română?"
# Expected documents:
docs15 = ["doc22.txt", "doc25.txt", "doc27.txt"]

# Test 16
query16 = "Ce poți spune despre Delta Dunării?"
# Expected documents:
docs16 = ["doc19.txt", "doc30.txt", "doc28.txt"]

queries = [query1, query2, query3, query4, query5, query6, query7, query8, query9, query10, query11, query12, query13,
           query14, query15, query16]
docs = [docs1, docs2, docs3, docs4, docs5, docs6, docs7, docs8, docs9, docs10, docs11, docs12, docs13, docs14, docs15,
        docs16]

# Test performance normal IR system

In [48]:
#### SEARCH QUERY #####
top1_correct = 0
top2_correct = 0
top3_correct = 0
total_queries = len(queries)

for query, doc in zip(queries, docs):
    results = ir_system.search(query)

    print(f"Query: {query}")
    print(f"IR results: {results}")
    print(f"Expected results: {doc}")

    # Compute Top-1 Accuracy
    top1_result = len(results) >= 1 and results[0] == doc[0]
    if top1_result:  # Check if the first result is in the expected results
        top1_correct += 1
    print(f"Top-1 Matches: {top1_result}")

    # compute Top-2 Accuracy
    top2_result = len(results) >= 2 and results[1] == doc[1] and top1_result
    if top2_result:
        top2_correct += 1
    print(f"Top-2 Matches: {top2_result}")

    # Compute Top-3 Accuracy
    top3_result = len(results) >= 3 and results[1] == doc[1] and top2_result

    if top3_result:
        top3_correct += 1
    print(f"Top-3 Matches: {top3_result}\n")

# Calculate accuracy
top1_accuracy = top1_correct / total_queries
top2_accuracy = top2_correct / total_queries
top3_accuracy = top3_correct / total_queries

# Print overall performance
print(f"Top-1 Accuracy: {top1_accuracy * 100:.2f}%")
print(f"Top-2 Accuracy: {top2_accuracy * 100:.2f}%")
print(f"Top-3 Accuracy: {top3_accuracy * 100:.2f}%")

Query: Ce pot vizita în România?
IR results: ['doc11.txt', 'doc19.txt', 'doc27.txt']
Expected results: ['doc11.txt', 'doc13.txt', 'doc19.txt']
Top-1 Matches: True
Top-2 Matches: False
Top-3 Matches: False

Query: Care sunt mâncărurile tradiționale românești?
IR results: ['doc17.txt']
Expected results: ['doc17.txt', 'doc27.txt', 'doc11.txt']
Top-1 Matches: True
Top-2 Matches: False
Top-3 Matches: False

Query: Cine sunt artiștii români celebri?
IR results: ['doc22.txt', 'doc25.txt', 'doc13.txt']
Expected results: ['doc22.txt', 'doc25.txt', 'doc27.txt']
Top-1 Matches: True
Top-2 Matches: True
Top-3 Matches: True

Query: Ce știi despre Castelul Bran și Dracula?
IR results: ['doc23.txt', 'doc17.txt', 'doc15.txt']
Expected results: ['doc23.txt', 'doc21.txt', 'doc19.txt']
Top-1 Matches: True
Top-2 Matches: False
Top-3 Matches: False

Query: Care este limbajul de programare utilizat în învățarea automată?
IR results: ['doc12.txt', 'doc20.txt', 'doc18.txt']
Expected results: ['doc12.txt', 'doc

# 1. Query Processing

In [49]:
def query_enhance(input_query="George Enescu"):
    chat = [
        {"role": "system",
         "content": "Ești un expert în procesarea interogărilor. Reformulează interogarea pentru a include subiecte relevante și posibile sinonime. Fii cât mai detaliat."},
        {"role": "user", "content": f"Interogare inițială: \"{input_query}\"\nReformulează interogarea:"},
    ]

    prompt = tokenizer.apply_chat_template(chat, tokenize=False, system_message="")
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to("cuda")

    # Create attention mask
    attention_mask = (inputs != tokenizer.pad_token_id).long()

    # Generate output
    outputs = model.generate(
        input_ids=inputs,
        attention_mask=attention_mask,
        max_new_tokens=128,
        pad_token_id=tokenizer.eos_token_id  # Ensure padding is treated as the end of a sequence
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Post-process the output to extract only the enhanced query
    if "assistant" in result:
        enhanced_query = result.split("assistant")[1].strip()
    else:
        enhanced_query = result.strip()

    return enhanced_query


query_enhance()

'Interogare reformulată: „George Enescu, un compozitor și muzician românesc, născut la 19 ianuarie 1881, în Iași, România. El este cunoscut pentru lucrările sale de muzică clasică, în special pentru compozițiile sale pentru orchestră și pian. Printre cele mai faimoase lucrări ale sale se numără „Concertul pentru vioară”, „Concertul pentru vioară și orchestră” și „Sextetul pentru vio'

# Test performance 1. Query Processing

In [50]:
# 1. Use the LLM the enhance the user's query
top1_correct = 0
top2_correct = 0
top3_correct = 0
total_queries = len(queries)

for idx in range(len(queries)):
    query = queries[idx]
    doc = docs[idx]
    enhanced_query = query_enhance(query)
    results = ir_system.search(enhanced_query)

    print(f"Query: {query}")
    print(f"IR results: {results}")
    print(f"Expected results: {doc}")

    # Compute Top-1 Accuracy
    top1_result = len(results) >= 1 and results[0] == doc[0]
    if top1_result:  # Check if the first result is in the expected results
        top1_correct += 1
    print(f"Top-1 Matches: {top1_result}")

    # compute Top-2 Accuracy
    top2_result = len(results) >= 2 and results[1] == doc[1] and top1_result
    if top2_result:
        top2_correct += 1
    print(f"Top-2 Matches: {top2_result}")

    # Compute Top-3 Accuracy
    top3_result = len(results) >= 3 and results[1] == doc[1] and top2_result

    if top3_result:
        top3_correct += 1
    print(f"Top-3 Matches: {top3_result}\n")

# Calculate accuracy
top1_accuracy = top1_correct / total_queries
top2_accuracy = top2_correct / total_queries
top3_accuracy = top3_correct / total_queries

# Print overall performance
print(f"Top-1 Accuracy: {top1_accuracy * 100:.2f}%")
print(f"Top-2 Accuracy: {top2_accuracy * 100:.2f}%")
print(f"Top-3 Accuracy: {top3_accuracy * 100:.2f}%")

Query: Ce pot vizita în România?
IR results: ['doc21.txt', 'doc27.txt', 'doc23.txt']
Expected results: ['doc11.txt', 'doc13.txt', 'doc19.txt']
Top-1 Matches: False
Top-2 Matches: False
Top-3 Matches: False

Query: Care sunt mâncărurile tradiționale românești?
IR results: ['doc17.txt', 'doc27.txt', 'doc12.txt']
Expected results: ['doc17.txt', 'doc27.txt', 'doc11.txt']
Top-1 Matches: True
Top-2 Matches: True
Top-3 Matches: True

Query: Cine sunt artiștii români celebri?
IR results: ['doc17.txt', 'doc15.txt', 'doc18.txt']
Expected results: ['doc22.txt', 'doc25.txt', 'doc27.txt']
Top-1 Matches: False
Top-2 Matches: False
Top-3 Matches: False

Query: Ce știi despre Castelul Bran și Dracula?
IR results: ['doc23.txt', 'doc27.txt', 'doc21.txt']
Expected results: ['doc23.txt', 'doc21.txt', 'doc19.txt']
Top-1 Matches: True
Top-2 Matches: False
Top-3 Matches: False

Query: Care este limbajul de programare utilizat în învățarea automată?
IR results: ['doc12.txt', 'doc20.txt', 'doc18.txt']
Expected

# 2. Answer Generation (LLM-Only)

In [51]:
import re


def search_combined_knowledge_base(input_query, documents_dict):
    """
    Combines all retrieved documents into a knowledge base and uses the LLM to return the top-3 most relevant documents.

    Args:
        input_query (str): The user's query.
        documents_dict (dict): A dictionary where keys are document names and values are document contents.
                               Example: {"Doc1": "Document 1 content", "Doc2": "Document 2 content", ...}

    Returns:
        list: Top-3 relevant documents, with document names and relevance explanations.
    """
    # Combine all documents into a single knowledge base string
    knowledge_base = "\n".join([f"Document: {doc_name}\nContent: {content}"
                                for doc_name, content in documents_dict.items()])

    # Create the prompt for the LLM
    chat = [
        {"role": "system",
         "content": (
             "Ești un expert în analizarea documentelor. Ai o bază de cunoștințe formată din documentele următoare."
             "Analizează baza de cunoștințe și întoarce primele 3 documente relevante pentru interogarea utilizatorului.")},
        {"role": "user",
         "content": f"Interogare: \"{input_query}\"\nBaza de cunoștințe:\n{knowledge_base}\n\nIdentifică cele mai relevante 3 documente și motivează răspunsul."},
    ]

    # Generate the prompt for the model
    prompt = tokenizer.apply_chat_template(chat, tokenize=False, system_message="")
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to("cuda")

    # Create attention mask
    attention_mask = (inputs != tokenizer.pad_token_id).long()

    # Generate output from the model
    outputs = model.generate(
        input_ids=inputs,
        attention_mask=attention_mask,
        max_new_tokens=512,  # Allow enough space for detailed responses
        pad_token_id=tokenizer.eos_token_id
    )
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Post-process the output to extract the top-3 documents
    if "assistant" in result:
        response = result.split("assistant")[1].strip()
    else:
        response = result.strip()

    top_documents = re.findall(r"(\w+\.txt)", response)

    return top_documents[:3]  # Return only the top-3 results

# Test performance 2. Answer Generation (LLM-Only)

In [52]:
import documents_generation

# 1. Use the LLM the enhance the user's query
top1_correct = 0
top2_correct = 0
top3_correct = 0
total_queries = len(queries)

for idx in range(len(queries)):
    query = queries[idx]
    doc = docs[idx]

    results = search_combined_knowledge_base(query, documents_generation.romanian_documents)

    print(f"Query: {query}")
    print(f"LLM results: {results}")
    print(f"Expected results: {doc}")

    # Compute Top-1 Accuracy
    top1_result = len(results) >= 1 and results[0] == doc[0]
    if top1_result:  # Check if the first result is in the expected results
        top1_correct += 1
    print(f"Top-1 Matches: {top1_result}")

    # compute Top-2 Accuracy
    top2_result = len(results) >= 2 and results[1] == doc[1] and top1_result
    if top2_result:
        top2_correct += 1
    print(f"Top-2 Matches: {top2_result}")

    # Compute Top-3 Accuracy
    top3_result = len(results) >= 3 and results[1] == doc[1] and top2_result

    if top3_result:
        top3_correct += 1
    print(f"Top-3 Matches: {top3_result}\n")

# Calculate accuracy
top1_accuracy = top1_correct / total_queries
top2_accuracy = top2_correct / total_queries
top3_accuracy = top3_correct / total_queries

# Print overall performance
print(f"Top-1 Accuracy: {top1_accuracy * 100:.2f}%")
print(f"Top-2 Accuracy: {top2_accuracy * 100:.2f}%")
print(f"Top-3 Accuracy: {top3_accuracy * 100:.2f}%")

Query: Ce pot vizita în România?
IR results: ['doc13.txt', 'doc19.txt', 'doc21.txt']
Expected results: ['doc11.txt', 'doc13.txt', 'doc19.txt']
Top-1 Matches: False
Top-2 Matches: False
Top-3 Matches: False

Query: Care sunt mâncărurile tradiționale românești?
IR results: ['doc17.txt', 'doc27.txt', 'doc29.txt']
Expected results: ['doc17.txt', 'doc27.txt', 'doc11.txt']
Top-1 Matches: True
Top-2 Matches: True
Top-3 Matches: True

Query: Cine sunt artiștii români celebri?
IR results: ['doc22.txt', 'doc25.txt', 'doc23.txt']
Expected results: ['doc22.txt', 'doc25.txt', 'doc27.txt']
Top-1 Matches: True
Top-2 Matches: True
Top-3 Matches: True

Query: Ce știi despre Castelul Bran și Dracula?
IR results: ['doc23.txt', 'doc21.txt', 'doc26.txt']
Expected results: ['doc23.txt', 'doc21.txt', 'doc19.txt']
Top-1 Matches: True
Top-2 Matches: True
Top-3 Matches: True

Query: Care este limbajul de programare utilizat în învățarea automată?
IR results: ['doc12.txt', 'doc18.txt', 'doc20.txt']
Expected resu